In [2]:
import pennylane as qml
import numpy as np
import warnings as warn
backend = qml.device('default.qubit', wires=6)
tol = 1E-6

In [1]:
# Prepares a|110000> + b|001100> with a^2 + b^2 = 1, a,b real valued
def StatePrep(a,b,c,d=1):
    r'''
    Prepares a circuit with the initial state:

    a|110000> + b|001100> + c|000011> + d|100100>

    where a, b, c, d are real valued and |(a,b,c,d)| = 1

    Args:
        a (float): a real valued parameter
        b (float): a real valued parameter
        c (float): a real valued parameter
        d (float): a real valued parameter
    
    Returns:
        qml.QubitStateVector: the desired initial state vector

    '''
    if np.abs(np.square(a) + np.square(b) + np.square(c) + np.square (d)- 1) > tol:
        if d == 1:
            pass
        else:
            warn.warn('Overdetermined parameters a and b: a^2 + b^2 + c^2 + d^2 != 1. Continuing with d = sqrt(1 - a^2 - b^2 - c^2)')
        d = np.sqrt(1 - np.square(a) - np.square(b) - np.square(c))
    
    qml.BasisState(np.array([0,0,1,1,0,0]), wires=[0,1,2,3,4,5])
    qml.DoubleExcitation(-1*np.arcsin(a)*2, wires=[2, 3, 0, 1])
    aBar = np.sqrt(1 - np.square(a))
    qml.DoubleExcitation(-1*np.arcsin(c/aBar)*2, wires=[2, 3, 4, 5])
    acBar = np.sqrt(1- np.square(a) - np.square(c))
    qml.ctrl(qml.SingleExcitation, control=3) (-1*np.arcsin(d/acBar)*2, wires=[2, 0])


    return qml.state()


def GenTestVals():
    r'''
    Generates a list of test values for the circuit by choosing a random vector and normalizing it.  

    May or may not be uniformly sampled - most likely does not randomly sample the unit hyper-circle

    Returns:
        tuple: a list of test values for the circuit (a,b,c,d)
    '''
    v = np.random.uniform(size=4)
    v = v / np.linalg.norm(v)
    print(v)
    return v[0],v[1],v[2],v[3]


In [3]:
# Generates the QNode for the circuit on the default simulated backend
circuit = qml.QNode(StatePrep, backend)

#Initial testing and manual verification
a,b,c,d = GenTestVals()
output = circuit(a,b,c,d=d)

# Strings the zero basis elements from the output state vector
states = [(output[i], np.binary_repr(i, width=6))
          for i in range(len(output)) if output[i] != 0]
print(states)


[0.74225916 0.06913616 0.64769335 0.15736856]
[(tensor(1.38777878e-17+0.j, requires_grad=True), '000100'), (tensor(0.96654357+0.j, requires_grad=True), '100011'), (tensor(0.25650249+0.j, requires_grad=True), '101100')]


In [4]:
def testValidInput():
    '''
    Tests the ideal conditions for the circuit - valid inputs
    '''
    a,b,c,d = GenTestVals()
    resultDict = {'110000':a, '001100':b, '000011': c, '100100':d}
    output = circuit(a, b, c, d=d)
    states = [(output[i], np.binary_repr(i, width=6))
                for i in range(len(output)) if output[i] != 0]
    for state in states:
        assert resultDict[state[1]] - state[0] < tol

#def test_InvalidD():

for i in range(10):
    testValidInput()


[0.74733813 0.54679436 0.3766585  0.02510007]


KeyError: '000100'